In [1]:
# prerequisites
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms
from torch.autograd import Variable
from torchvision.utils import save_image

bs = 100
# MNIST Dataset
train_dataset = datasets.MNIST(root='./mnist_data/', train=True, transform=transforms.ToTensor(), download=True)
test_dataset = datasets.MNIST(root='./mnist_data/', train=False, transform=transforms.ToTensor(), download=False)

# Data Loader (Input Pipeline)
train_loader = torch.utils.data.DataLoader(dataset=train_dataset, batch_size=bs, shuffle=True)
test_loader = torch.utils.data.DataLoader(dataset=test_dataset, batch_size=bs, shuffle=False)

100.0%


Extracting ./mnist_data/MNIST\raw\train-images-idx3-ubyte.gz to ./mnist_data/MNIST\raw


100.0%


Extracting ./mnist_data/MNIST\raw\train-labels-idx1-ubyte.gz to ./mnist_data/MNIST\raw




63.6%

100.0%
100.0%

Extracting ./mnist_data/MNIST\raw\t10k-images-idx3-ubyte.gz to ./mnist_data/MNIST\raw

Extracting ./mnist_data/MNIST\raw\t10k-labels-idx1-ubyte.gz to ./mnist_data/MNIST\raw



In [3]:
class VAE(nn.Module):
    def __init__(self, x_dim, h_dim1, h_dim2, z_dim):
        super(VAE, self).__init__()
        
        # encoder part
        self.fc1 = nn.Linear(x_dim, h_dim1)
        self.fc2 = nn.Linear(h_dim1, h_dim2)
        self.fc31 = nn.Linear(h_dim2, z_dim)
        self.fc32 = nn.Linear(h_dim2, z_dim)
        # decoder part
        self.fc4 = nn.Linear(z_dim, h_dim2)
        self.fc5 = nn.Linear(h_dim2, h_dim1)
        self.fc6 = nn.Linear(h_dim1, x_dim)
        
    def encoder(self, x):
        h = F.relu(self.fc1(x))
        h = F.relu(self.fc2(h))
        return self.fc31(h), self.fc32(h) # mu, log_var
    
    def sampling(self, mu, log_var):
        std = torch.exp(0.5*log_var)
        eps = torch.randn_like(std)
        return eps.mul(std).add_(mu) # return z sample
        
    def decoder(self, z):
        h = F.relu(self.fc4(z))
        h = F.relu(self.fc5(h))
        return F.sigmoid(self.fc6(h)) 
    
    def forward(self, x):
        mu, log_var = self.encoder(x.view(-1, 784))
        z = self.sampling(mu, log_var)
        return self.decoder(z), mu, log_var

# build model
vae = VAE(x_dim=784, h_dim1= 512, h_dim2=256, z_dim=2)
if torch.cuda.is_available():
    vae.cuda()

In [4]:
vae

VAE(
  (fc1): Linear(in_features=784, out_features=512, bias=True)
  (fc2): Linear(in_features=512, out_features=256, bias=True)
  (fc31): Linear(in_features=256, out_features=2, bias=True)
  (fc32): Linear(in_features=256, out_features=2, bias=True)
  (fc4): Linear(in_features=2, out_features=256, bias=True)
  (fc5): Linear(in_features=256, out_features=512, bias=True)
  (fc6): Linear(in_features=512, out_features=784, bias=True)
)

In [5]:
optimizer = optim.Adam(vae.parameters())
# return reconstruction error + KL divergence losses
def loss_function(recon_x, x, mu, log_var):
    BCE = F.binary_cross_entropy(recon_x, x.view(-1, 784), reduction='sum')
    KLD = -0.5 * torch.sum(1 + log_var - mu.pow(2) - log_var.exp())
    return BCE + KLD

In [6]:
def train(epoch):
    vae.train()
    train_loss = 0
    for batch_idx, (data, _) in enumerate(train_loader):
        data = data.cuda()
        optimizer.zero_grad()
        
        recon_batch, mu, log_var = vae(data)
        loss = loss_function(recon_batch, data, mu, log_var)
        
        loss.backward()
        train_loss += loss.item()
        optimizer.step()
        
        if batch_idx % 100 == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, batch_idx * len(data), len(train_loader.dataset),
                100. * batch_idx / len(train_loader), loss.item() / len(data)))
    print('====> Epoch: {} Average loss: {:.4f}'.format(epoch, train_loss / len(train_loader.dataset)))

In [7]:
def test():
    vae.eval()
    test_loss= 0
    with torch.no_grad():
        for data, _ in test_loader:
            data = data.cuda()
            recon, mu, log_var = vae(data)
            
            # sum up batch loss
            test_loss += loss_function(recon, data, mu, log_var).item()
        
    test_loss /= len(test_loader.dataset)
    print('====> Test set loss: {:.4f}'.format(test_loss))

In [8]:
for epoch in range(1, 51):
    train(epoch)
    test()

Train Epoch: 1 [0/60000 (0%)]	Loss: 544.488516
Train Epoch: 1 [10000/60000 (17%)]	Loss: 196.735410
Train Epoch: 1 [20000/60000 (33%)]	Loss: 172.674219
Train Epoch: 1 [30000/60000 (50%)]	Loss: 171.841094
Train Epoch: 1 [40000/60000 (67%)]	Loss: 165.331387
Train Epoch: 1 [50000/60000 (83%)]	Loss: 166.431172
====> Epoch: 1 Average loss: 179.2632
====> Test set loss: 162.4878
Train Epoch: 2 [0/60000 (0%)]	Loss: 160.239336
Train Epoch: 2 [10000/60000 (17%)]	Loss: 153.542705
Train Epoch: 2 [20000/60000 (33%)]	Loss: 169.893223
Train Epoch: 2 [30000/60000 (50%)]	Loss: 151.379014
Train Epoch: 2 [40000/60000 (67%)]	Loss: 157.035879
Train Epoch: 2 [50000/60000 (83%)]	Loss: 150.314355
====> Epoch: 2 Average loss: 158.0539
====> Test set loss: 154.7595
Train Epoch: 3 [0/60000 (0%)]	Loss: 151.995391
Train Epoch: 3 [10000/60000 (17%)]	Loss: 150.184961
Train Epoch: 3 [20000/60000 (33%)]	Loss: 146.723730
Train Epoch: 3 [30000/60000 (50%)]	Loss: 160.486748
Train Epoch: 3 [40000/60000 (67%)]	Loss: 153.92

Train Epoch: 22 [50000/60000 (83%)]	Loss: 142.247852
====> Epoch: 22 Average loss: 138.0976
====> Test set loss: 139.7158
Train Epoch: 23 [0/60000 (0%)]	Loss: 149.485615
Train Epoch: 23 [10000/60000 (17%)]	Loss: 143.120107
Train Epoch: 23 [20000/60000 (33%)]	Loss: 144.853584
Train Epoch: 23 [30000/60000 (50%)]	Loss: 133.426406
Train Epoch: 23 [40000/60000 (67%)]	Loss: 134.192627
Train Epoch: 23 [50000/60000 (83%)]	Loss: 145.757236
====> Epoch: 23 Average loss: 137.7248
====> Test set loss: 139.2135
Train Epoch: 24 [0/60000 (0%)]	Loss: 140.885830
Train Epoch: 24 [10000/60000 (17%)]	Loss: 145.449580
Train Epoch: 24 [20000/60000 (33%)]	Loss: 142.010938
Train Epoch: 24 [30000/60000 (50%)]	Loss: 141.315400
Train Epoch: 24 [40000/60000 (67%)]	Loss: 145.445039
Train Epoch: 24 [50000/60000 (83%)]	Loss: 136.647354
====> Epoch: 24 Average loss: 137.5859
====> Test set loss: 139.3073
Train Epoch: 25 [0/60000 (0%)]	Loss: 138.812734
Train Epoch: 25 [10000/60000 (17%)]	Loss: 138.111865
Train Epoch: 

Train Epoch: 44 [20000/60000 (33%)]	Loss: 138.032021
Train Epoch: 44 [30000/60000 (50%)]	Loss: 128.380205
Train Epoch: 44 [40000/60000 (67%)]	Loss: 134.308994
Train Epoch: 44 [50000/60000 (83%)]	Loss: 128.784414
====> Epoch: 44 Average loss: 134.7829
====> Test set loss: 138.0945
Train Epoch: 45 [0/60000 (0%)]	Loss: 132.146328
Train Epoch: 45 [10000/60000 (17%)]	Loss: 137.589570
Train Epoch: 45 [20000/60000 (33%)]	Loss: 131.492383
Train Epoch: 45 [30000/60000 (50%)]	Loss: 130.970703
Train Epoch: 45 [40000/60000 (67%)]	Loss: 135.645000
Train Epoch: 45 [50000/60000 (83%)]	Loss: 129.068252
====> Epoch: 45 Average loss: 134.4284
====> Test set loss: 138.2794
Train Epoch: 46 [0/60000 (0%)]	Loss: 135.000918
Train Epoch: 46 [10000/60000 (17%)]	Loss: 132.472471
Train Epoch: 46 [20000/60000 (33%)]	Loss: 127.018945
Train Epoch: 46 [30000/60000 (50%)]	Loss: 125.478994
Train Epoch: 46 [40000/60000 (67%)]	Loss: 132.799619
Train Epoch: 46 [50000/60000 (83%)]	Loss: 137.282988
====> Epoch: 46 Average 

In [9]:
with torch.no_grad():
    z = torch.randn(64, 2).cuda()
    sample = vae.decoder(z).cuda()
    
    save_image(sample.view(64, 1, 28, 28), './samples/sample_3' + '.png')